In [ ]:
pip install duckduckgo-search

In [106]:
import duckduckgo_search

import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [120]:
number_of_pages = 8

In [121]:
def company_details(url :str):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    rows = soup.find_all('tr')
    
    list_company_name = []
    list_company_id = []
    list_company_address = []
    list_person_name = []
    list_person_address = []
    list_detail_link = []
    list_pdf_link = []
    
    for row in rows:
        data = row.find_all('td')
        if len(data) > 0:
            status = data[2].text.strip()
            if status == 'Active':
                detail = data[0].find("a")['href']
                detail_link = 'https://search.sunbiz.org/' + detail

                response_details = requests.get(detail_link)
                soup_details = BeautifulSoup(response_details.content, 'html.parser')

                section = soup_details.find_all("div", class_="detailSection")[2]
                #print(detail_link)
                if section.find("div") != None:
                    address = " ".join(section.find("div").stripped_strings)
                    #print(address)
                    
                try:
                    section_name = soup_details.find_all("div", class_="detailSection")[5]
                    #print(detail_link)
                    if section_name.find("div") != None:
                        person_name = section_name.contents[14]
                        #print(person_name)
                        person_address = "".join(section_name.find("div").stripped_strings)
                        #print(person_address)
                except IndexError:
                    person_address = 'null'
                    person_name = "null"

                try:
                    section = soup_details.find_all("div", class_="detailSection")[7]
                    pdf = section.find_all("td")[0].find("a")['href']
                    pdf_link = 'https://search.sunbiz.org/' + pdf
                except TypeError:
                    pdf_link = "null"
                except IndexError:
                    continue

                company_name = data[0].text.strip()
                company_id = data[1].text.strip()
                
                
                list_company_name.append(company_name)
                list_company_id.append(company_id)
                list_company_address.append(address)
                list_person_address.append(person_address)
                list_detail_link.append(detail_link)
                list_person_name.append(person_name)
                list_pdf_link.append(pdf_link)


    return list_company_name, list_company_id, list_company_address, list_person_address, list_detail_link, list_person_name, list_pdf_link

In [122]:
def if_similar(statement):
  results = duckduckgo_search.ddg(statement)[:10]
  benchmark = 0.25
  for result in results:
    statement2 = result['title']
    score = calculate_similarity(statement, statement2)

    if score > benchmark:
      return "available"
    else:
      return "not available"

In [123]:
def calculate_similarity(sentence1, sentence2):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([sentence1, sentence2])
    similarity = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])
    return similarity[0][0]

In [124]:
stop = number_of_pages
nameOrder = None
next_page = []
for page in range(0,stop):
    
    if page == 0:
        url  = f'https://search.sunbiz.org/Inquiry/CorporationSearch/SearchResults/EntityName/cake/Page1?searchNameOrder=CAKE'
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        rows = soup.find_all('tr')
        data = rows[19].find_all('td')
        if len(data) > 0:
            nameOrder = data[0].find("a")['href'].split('&')[2].split('=')[1]
            next_page.append(nameOrder)
            print(nameOrder)
    else:
        url = f'https://search.sunbiz.org/Inquiry/CorporationSearch/SearchResults?InquiryType=EntityName&inquiryDirectionType=ForwardList&searchNameOrder={nameOrder}&SearchTerm=cake'
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        rows = soup.find_all('tr')
        data = rows[19].find_all('td')
        if len(data) > 0:
            nameOrder = data[0].find("a")['href'].split('&')[2].split('=')[1]
            next_page.append(nameOrder)
            print(nameOrder)
            
print("Done...")

CAKEAHOLIC%20L200002634040
CAKEBAKE%20L180000074960
CAKEBOUTIQUE%20L040000627090
CAKEBURGER%20P180000213550
CAKECARTEL%20P170000478940
CAKECOUTURE%20L120001098100
CAKECREATIVITY%20P050001298850
CAKEDBYGG%20L220005137380
Done...


In [125]:
corporation = []
corporation_id = []
corporation_address = []
person_name = []
person_address = []
detail_link = []
pdf_link = []

start_page = 0
end_page = stop
for page_no in range(start_page, end_page):
    
    if page_no == 0:
    
        print(f'starting scraping for page:{page_no}')

        url  = f'https://search.sunbiz.org/Inquiry/CorporationSearch/SearchResults/EntityName/cake/Page{page_no}?searchNameOrder=CAKE'
        company_name_, company_id_, address_, person_address_, detail_link_, person_name_, pdf_link_ = company_details(url)

        corporation.extend(company_name_)
        corporation_id.extend(company_id_)
        corporation_address.extend(address_)
        person_address.extend(person_address_)
        detail_link.extend(detail_link_)
        person_name.extend(person_name_)
        pdf_link.extend(pdf_link_)

        print(f'finished scraping page:{page_no}')
        
    else:
        
        print(f'starting scraping for page:{page_no}')

        url  = f'https://search.sunbiz.org/Inquiry/CorporationSearch/SearchResults?InquiryType=EntityName&inquiryDirectionType=ForwardList&searchNameOrder={next_page[page_no-1]}&SearchTerm=cake'
        company_name_, company_id_, address_, person_address_, detail_link_, person_name_, pdf_link_ = company_details(url)

        corporation.extend(company_name_)
        corporation_id.extend(company_id_)
        corporation_address.extend(address_)
        person_address.extend(person_address_)
        detail_link.extend(detail_link_)
        person_name.extend(person_name_)
        pdf_link.extend(pdf_link_)
        print(f'finished scraping page:{page_no}')

print("scraping successful")

starting scraping for page:0
finished scraping page:0
starting scraping for page:1
finished scraping page:1
starting scraping for page:2
finished scraping page:2
starting scraping for page:3
finished scraping page:3
starting scraping for page:4
finished scraping page:4
starting scraping for page:5
finished scraping page:5
starting scraping for page:6
finished scraping page:6
starting scraping for page:7
finished scraping page:7
scraping successful


In [126]:
df = pd.DataFrame({"company_name": corporation, "company_id":corporation_id, "status":"Active", "corporation_address":corporation_address, "name":person_name, "address":person_address, "detail_links":detail_link, "pdf_link":pdf_link})
df['name'] = df['name'].apply(lambda row: row.replace("\r\n","").replace("  ", ""))
df = df.replace("null",np.NaN).dropna()
df = df.drop(columns=['company_id','detail_links'])
df['company_name'] + df['corporation_address']
df['name_address'] = df['company_name'] + df['corporation_address']

In [127]:
df['results'] = df['name_address'].apply(lambda row: if_similar(row))

/usr/local/lib/python3.10/dist-packages/duckduckgo_search/compat.py:20: UserWarning: ddg is deprecated. Use DDGS().text() generator
  warnings.warn("ddg is deprecated. Use DDGS().text() generator")
/usr/local/lib/python3.10/dist-packages/duckduckgo_search/compat.py:24: UserWarning: parameter page is deprecated, use DDGS().text() generator
  warnings.warn("parameter page is deprecated, use DDGS().text() generator")


In [128]:
df = df.drop(columns=['name_address'])

In [129]:
df

,company_name,status,corporation_address,name,address,pdf_link,results
0,CAKE CORPORATION,Active,"201 N Franklin St Suite 1000 Tampa, FL 33602","Bennett, Bruce","201 N Franklin StSuite 1000Tampa, FL 33602",https://search.sunbiz.org//Inquiry/Corporation...,not available
1,"CAKE & COMPANY, LLC",Active,"6315 4TH LN VERO BEACH, FL 32968","ZAMBRANO, GABRIELA","6315 4TH LNVERO BEACH, FL 32968",https://search.sunbiz.org//Inquiry/Corporation...,available
2,CAKE 24/7 LLC,Active,"1403 TIFTON ST SOUTH SAINT PETERSBURG, FL 33711","TAYLOR, SHARQUETTA T","1403 TIFTON ST SOUTHSAINT PETERSBURG, FL 33711",https://search.sunbiz.org//Inquiry/Corporation...,available
3,CAKE AND COMPANY 321 LLC,Active,"1105 CORONADO DR ROCKLEDGE, FL 32955","RAIT, ASHLEY M","1105 CORONADO DRROCKLEDGE, FL 32955",https://search.sunbiz.org//Inquiry/Corporation...,available
4,CAKE 4 PARTNERS INC.,Active,"13470-3 Beach Blvd. JACKSONVILLE, FL 32224","WATERS, CHARLES","13470-3 Beach Blvd.JACKSONVILLE, FL 32224",https://search.sunbiz.org//Inquiry/Corporation...,available
5,"CAKE8, LLC",Active,"128 Salem Street Altamonte Springs, FL 32701","HARRIS, SIVI","128 Salem StreetAltamonte Springs, FL 32701",https://search.sunbiz.org//Inquiry/Corporation...,available
6,"CAKE ADVERTISING, LLC",Active,"3750 CRAWLEY DOWN LOOP SANFORD, FL 32773","ALATARI, JENNIFER","3750 CRAWLEY DOWN LOOPSANFORD, FL 32773",https://search.sunbiz.org//Inquiry/Corporation...,available
7,CAKE AFFECTION INC.,Active,"4691 100th Way N #b SAINT PETERSBURG, FL 33708","LIPPARD, AARON","4691 100th Way N #bSAINT PETERSBURG, FL 33708",https://search.sunbiz.org//Inquiry/Corporation...,not available
8,"CAKE AH LICIOUS, LLC",Active,"12314 MELROSE WAY BOCA RATON, FL 33428","GLASSER, CELIA R","12314 MELROSE WAYBOCA RATON, FL 33428",https://search.sunbiz.org//Inquiry/Corporation...,available
9,CAKEAHOLIC LLC,Active,"47 PINE HILL LANE PALM COAST, FL 32164","JACKSON, SHEERA T","47 PINE HILL LNPALM COAST, FL 32164",https://search.sunbiz.org//Inquiry/Corporation...,available


In [119]:
df.to_excel('scraped_data.xlsx', index=False)